# Survival analysis with Catboost

Unlike common supervised regression task where target variable is known and observed during the entire period in training dataset, survival regression has some limitations in terms of partially observed or **censored** target variable.  
**Censoring** is a form of missing data problem in which time to event is not observed for reasons such as termination of study before all recruited subjects have shown the event of interest or the subject has left the study prior to experiencing an event.  
  
Censoring types:
 * **Right-censored:** When the patient is lost to followup or the event does not occur within the study duration. 
 * **Left-censored:** When the patient had been on risk for disease for a period before entering the study.
 * **Interval-censored:** When time to event may be known only up to a time interval.
 * **Uncensored:** When everything is alright:)

## Cox proportional model

Survival models of this type can be viewed as consisting of two parts: the underlying <a href="https://en.wikipedia.org/wiki/Hazard_function"> baseline hazard </a> function, often denoted ${\displaystyle \lambda _{0}(t)}$, describing how the risk of event per time unit changes over time at baseline levels of covariates; and the effect parameters, describing how the hazard varies in response to explanatory covariates. 

Let $X_i=(X_{i1},...,X_{ip})$ be the realized values of the covariates for subject i. The hazard function for the Cox proportional hazards model has the form:

$$\lambda(t|X_i)=\lambda_0(t)exp(h(X_i)),$$
where $h(X_i)$ is the hazard rate for considered sample $X_i$, represented as CatBoost model.This expression gives the hazard function at time $t$ for subject $i$ with covariate vector (explanatory variables) $X_i$.The likelihood of the event to be observed occurring for subject $i$ at time $Y_i$ can be written as:

$$L_i(\beta)=
    \frac{\lambda(Y_i|X_i)}{\sum_{j: Y_j \ge Y_i}\lambda(Y_i|X_j)} =
    \frac{\lambda_0(Y_i)exp(h(X_i))}{\sum_{j: Y_j \ge Y_i}\lambda_0(Y_i)exp(h(X_j))} =
    \frac{exp(h(X_i))}{\sum_{j: Y_j \ge Y_i}exp(h(X_j))}
$$

Treating the subjects as if they were statistically independent of each other, the joint probability of all realized events is the following partial likelihood, where the occurrence of the event is indicated by $C_i = 1$:

$${\displaystyle L(\beta )=\prod _{i:C_{i}=1}L_{i}(\beta ).}$$

The corresponding log partial likelihood is:

$$\ell(\beta)=\sum_{i:C_i=1}(h(X_i) - log \sum_{j:Y_j \ge Y_i} exp(h(X_j)))$$

This is the function we are trying to optimize via CatBoost.

### import data

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#dataset: https://scikit-survival.readthedocs.io/en/latest/api/generated/sksurv.datasets.load_flchain.html#sksurv.datasets.load_flchain

In [9]:
data = pd.read_csv("untitled.txt", header=None, names=[
    'age', 
    'sex', 
    'sample_year', 
    'kappa', 
    'lambda', 
    'flc.grp', 
    'creatinine',
    'mgus',
    'futime',
    'death', 
    'chapter',
])
data['label'] = np.where(data['death'] == 'dead', data['futime'], - data['futime'])

In [25]:
data.head(10)

,age,sex,sample_year,kappa,lambda,flc.grp,creatinine,mgus,futime,death,chapter,label
0,97,F,1997,5.700,4.860,10,1.7,no,85,dead,Circulatory,85
1,92,F,2000,0.870,0.683,1,0.9,no,1281,dead,Neoplasms,1281
2,94,F,1997,4.360,3.850,10,1.4,no,69,dead,Circulatory,69
3,92,F,1996,2.420,2.220,9,1,no,115,dead,Circulatory,115
4,93,F,1996,1.320,1.690,6,1.1,no,1039,dead,Circulatory,1039
5,90,F,1997,2.010,1.860,9,1,no,1355,dead,Mental,1355
6,90,F,1996,0.430,0.880,1,0.8,no,2851,dead,Mental,2851
7,90,F,1999,2.470,2.700,10,1.2,no,372,dead,Nervous,372
8,93,F,1996,1.910,2.180,9,1.2,no,3309,dead,Respiratory,3309
9,91,F,1996,0.791,2.220,6,0.8,no,1326,dead,Circulatory,1326


In [26]:
train, test = train_test_split(data)

In [18]:
features = ['age', 'sex', 'sample_year', 'kappa', 'lambda', 'flc.grp', 'creatinine', 'mgus', 'chapter']
cat_features = ['sex', 'mgus', 'creatinine', 'chapter']

In [8]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

In [29]:
train_pool = Pool(train[features], label=train['label'], cat_features=cat_features)
test_pool = Pool(test[features], label=test['label'], cat_features=cat_features)

### Fit model

In [22]:
model = CatBoostRegressor(iterations=10,
                          learning_rate=1,
                          depth=2,
                          loss_function='Cox',
                          eval_metric='Cox'
                         )

In [30]:
model.fit(train_pool, eval_set=test_pool)

0:	learn: 13381.0307883	test: 3663.4413570	best: 3663.4413570 (0)	total: 5.96ms	remaining: 53.6ms
1:	learn: 12991.8923950	test: 3537.8381191	best: 3537.8381191 (1)	total: 11.2ms	remaining: 44.9ms
2:	learn: 12656.9183918	test: 3429.0640358	best: 3429.0640358 (2)	total: 15.9ms	remaining: 37.1ms
3:	learn: 12380.5311749	test: 3339.6929800	best: 3339.6929800 (3)	total: 20.8ms	remaining: 31.1ms
4:	learn: 12146.8691439	test: 3264.4730846	best: 3264.4730846 (4)	total: 25.4ms	remaining: 25.4ms
5:	learn: 11955.7733316	test: 3202.4297125	best: 3202.4297125 (5)	total: 30.4ms	remaining: 20.3ms
6:	learn: 11802.1765372	test: 3152.0366628	best: 3152.0366628 (6)	total: 35.1ms	remaining: 15ms
7:	learn: 11671.1212869	test: 3109.2440195	best: 3109.2440195 (7)	total: 39.8ms	remaining: 9.94ms
8:	learn: 11561.4427202	test: 3073.4239605	best: 3073.4239605 (8)	total: 44.5ms	remaining: 4.95ms
9:	learn: 11476.3890208	test: 3044.8369522	best: 3044.8369522 (9)	total: 49.4ms	remaining: 0us

bestTest = 3044.836952
b

In [31]:
model.predict(test_pool)

array([-1.26033152, -1.27241036,  1.05641255, ..., -1.27241036,
       -1.20344912,  1.05664301])

### Accelerated Failure Time (AFT) model 

Base parametric model  
$$\log T_i = \langle w, x_i \rangle + \epsilon_i$$  
$T_i$ - survival time of the i-th subject
$w$ - learnable parameters  
$x_i$ - subject time-invariant feature description  
$\epsilon_i$ - error term with. $\epsilon_i \sim \sigma Z_i$ where $Z_i$ is a prespecified a distribution and $\sigma$ is a scaling hyperparamater.

Idea - replace linear term $\langle w, x \rangle$ with boosted decision trees classifier $H(x)$.

### Fitting process 

As $T_i$ is random variable, we can utilize common approach - maximize likelihood function across $n$ observed subjects $$L= \prod_{i=1}^n f_T(t=T_i) \sim \sum_{i=1}^n \log f_T(t=T_i)$$ where $f_T(t)$ is a PDF of $T = g(\epsilon) = e^{H(x)+\epsilon}$ prespecified distribution's PDF.  
With an eye on interval form of censored target, likelihood function can be decomposed into two parts:   
a) likelihood for uncensored data  
b) likelihood for arbitrary censored data  
$$\log L = \sum_{i=1}^n \log f_T(t=T_i) = \sum_{a} \log f_T(t=T_a) + \sum_{b} \log (F_T(t=\overline{T_b})-F_T(t=\underline{T_{b}}))$$
where $F_T(t)$ is a prespecified distribution's CDF  
$\overline{T}, \underline{T}$ - upper and lower bounds of the interval  


In Catboost Z can be distributed according to one of three distributions:

| Distribution | CDF | PDF |
| --- | --- | --- |
| Normal |$$\dfrac{1}{2}(1+erf( \dfrac{z}{\sqrt{2}})) $$ | $$\dfrac{e^{\dfrac{-z^2}{2}}}{\sqrt{2\pi}}$$|
| Logistic | $$\dfrac{e^z}{1+e^z}$$| $$\dfrac{e^z}{(1+e^z)^2}$$ |
| Extreme | $$1-e^{-e^z}$$  | $$e^ze^{-e^z}$$  |

In order to use censored data in Catboost one should express target variable in interval form:
 * **Right-censored:** $[ A, +\infty ]$
 * **Left-censored:** $[0, B]$
 * **Interval-censored:** $[A, B]$
 * **Uncensored:** $[A, A]$  
where $A$, $B$ - finite non-negative numbers

### Fit model 

#### Load dataset 

We are going to use [WHAS500* dataset](https://github.com/sebp/scikit-survival/tree/master/sksurv/datasets/data) and will try to predict patients' lifetime after the hospital admission.  
*Table 1.2
of Hosmer, D.W. and Lemeshow, S. and May, S. (2008) Applied Survival
Analysis: Regression Modeling of Time to Event Data: Second Edition,
John Wiley and Sons Inc., New York, NY

In [3]:
data = pd.read_csv('whas500.csv')

In order to use right-censored data in catboost use -1 to express $+\infty$. 

In [4]:
data['y_lower'] = data['lenfol']
data['y_upper'] = np.where(data['fstat'], data['lenfol'], -1)

In [5]:
statifying_column = data['fstat']
data = data.drop(['fstat','lenfol'],axis=1)

In [6]:
train, test = train_test_split(data, test_size=0.2, stratify=statifying_column)

In [7]:
data.head(5)

,age,gender,hr,sysbp,diasbp,bmi,cvd,afb,sho,chf,av3,miord,mitype,los,y_lower,y_upper
0,83.0,0,89.0,152.0,78.0,25.54051,1,1,0,0,0,1,0,5.0,2178.0,-1.0
1,49.0,0,84.0,120.0,60.0,24.02398,1,0,0,0,0,0,1,5.0,2172.0,-1.0
2,70.0,1,83.0,147.0,88.0,22.14290,0,0,0,0,0,0,1,5.0,2190.0,-1.0
3,70.0,0,65.0,123.0,76.0,26.63187,1,0,0,1,0,0,1,10.0,297.0,297.0
4,70.0,0,63.0,135.0,85.0,24.41255,1,0,0,0,0,0,1,6.0,2131.0,-1.0


In [8]:
features = data.columns.difference(['y_lower', 'y_upper'], sort=False)
cat_features = ['cvd','afb','sho','chf','av3', 'miord', 'mitype', 'gender']

In [9]:
train_pool = Pool(train[features], label=train.loc[:,['y_lower','y_upper']], cat_features=cat_features)
test_pool = Pool(test[features], label=test.loc[:,['y_lower','y_upper']], cat_features=cat_features)

loss function format: SurvivalAft:dist={distname}scale={int}  
Hyperparameters:
* distname $\in$ [Normal, Logistic, Extreme]
* scale - any positive number representing distribution scale

eval metric: SurvivalAft   
Metric is calculated as "interval MAE":  
$$\text{Interval MAE} = \begin{cases}
0 & \text{if $\hat{y} \in [\underline{y}, \overline{y}]$}\\
\min{(|\hat{y}-\underline{y}|,|\hat{y}-\overline{y}|)} & \text{if $\hat{y} \notin [\underline{y}, \overline{y}]$}
\end{cases}$$


In [88]:
model_normal = CatBoostRegressor(iterations=500,
                                 loss_function='SurvivalAft:dist=Normal',
                                 eval_metric='SurvivalAft',
                                 verbose=0
                                )

In [89]:
model_logistic = CatBoostRegressor(iterations=500,
                                 loss_function='SurvivalAft:dist=Logistic;scale=1.2',
                                 eval_metric='SurvivalAft',
                                 verbose=0)

In [90]:
model_extreme = CatBoostRegressor(iterations=500,
                                 loss_function='SurvivalAft:dist=Extreme;scale=2',
                                 eval_metric='SurvivalAft',
                                 verbose=0)

In [91]:
_ = model_normal.fit(train_pool, eval_set=test_pool)
_ = model_logistic.fit(train_pool, eval_set=test_pool)
_ = model_extreme.fit(train_pool, eval_set=test_pool)

In [92]:
train_predictions = pd.DataFrame({'y_lower': train['y_lower'],
                                  'y_upper': train['y_upper'],
                                  'preds_normal': model_normal.predict(train_pool, prediction_type='Exponent'),
                                  'preds_logistic': model_logistic.predict(train_pool, prediction_type='Exponent'),
                                  'preds_extreme': model_extreme.predict(train_pool, prediction_type='Exponent')})
train_predictions['y_upper'] = np.where(train_predictions['y_upper']==-1, np.inf, train_predictions['y_upper'])

test_predictions = pd.DataFrame({'y_lower': test['y_lower'],
                                  'y_upper': test['y_upper'],
                                  'preds_normal': model_normal.predict(test_pool, prediction_type='Exponent'),
                                  'preds_logistic': model_logistic.predict(test_pool, prediction_type='Exponent'),
                                  'preds_extreme': model_extreme.predict(test_pool, prediction_type='Exponent')})
test_predictions['y_upper'] = np.where(test_predictions['y_upper']==-1, np.inf, test_predictions['y_upper'])

### Evaluate metric 

In [93]:
def interval_mae(y_true_lower, y_true_upper, y_pred):
    mae = np.where((y_true_lower <= y_pred) & (y_pred <= y_true_upper),
                   0,
                   np.minimum(np.abs(y_true_lower-y_pred),
                              np.abs(y_true_upper-y_pred))) 
    return mae.mean()

In [94]:
distributions = ['normal', 'logistic', 'extreme']
print('Interval MAE')
for dist in distributions:
    train_metric = interval_mae(train_predictions['y_lower'], train_predictions['y_upper'], train_predictions[f'preds_{dist}'])
    test_metric = interval_mae(test_predictions['y_lower'], test_predictions['y_upper'], test_predictions[f'preds_{dist}'])
    print(f'Train set. dist:{dist}: {train_metric:0.2f}')
    print(f'Test set. dist:{dist}: {test_metric:0.2f}')
    print('---------------------------')

Interval MAE
Train set. dist:normal: 171.70
Test set. dist:normal: 286.21
---------------------------
Train set. dist:logistic: 404.54
Test set. dist:logistic: 348.71
---------------------------
Train set. dist:extreme: 323.84
Test set. dist:extreme: 296.83
---------------------------
